# Objetivo: 
 > Distinguir Series vs DataFrame, máscaras y cuándo iterar.

# 0) Setup: dataset de ejemplo: 'customers'


In [2]:
import pandas as pd
import numpy as np

rng = np.random.default_rng(7)

N = 500
segments = ['SMB','MidMarket','Enterprise','Consumer']
regions  = ['EU','NA','LATAM','APAC']
channels = ['Online','Retail','Partner']

customers = pd.DataFrame({
    'customer_id': np.arange(1, N+1),
    'segment': rng.choice(segments, size=N, p=[0.35,0.25,0.15,0.25]),
    'region': rng.choice(regions, size=N),
    'channel_preferred': rng.choice(channels, size=N, p=[0.6,0.25,0.15]),
    'created_at': pd.to_datetime('2023-01-01') + pd.to_timedelta(rng.integers(0, 600, size=N), unit='D')
})
customers['b2_type'] = np.where(customers['segment'].eq('Consumer'), 'B2C', 'B2B')

display(customers.head())

,customer_id,segment,region,channel_preferred,created_at,b2_type
0,1,Enterprise,APAC,Online,2023-04-22,B2B
1,2,Consumer,NA,Retail,2024-06-22,B2C
2,3,Consumer,NA,Online,2024-05-11,B2C
3,4,SMB,APAC,Online,2023-03-15,B2B
4,5,SMB,NA,Online,2024-08-14,B2B


# 1) Series vs DataFrame
- **Series**: una **columna** con índice.
- **DataFrame**: una **tabla** (1..N columnas).

👉 *Regla práctica*: usa **Series** para crear **máscaras** o hacer operaciones por columna; usa **DataFrame** cuando quieras devolver/seguir trabajando con **tabla** (selección de varias columnas, `merge`, exportación…).

In [3]:
customers["segment"]
type(customers["segment"])

pandas.core.series.Series

In [4]:
customers[["segment"]]
type(customers[["segment"]])

pandas.core.frame.DataFrame

# 2) ¿Qué es cada cosa y para qué sirve?

## Serie (pd.Series)

- Es una columna con índice.
- Ej.: customers["region"] → Series (dtype: object).
- Cuándo: cálculos por columna, máscaras booleanas, stats rápidos.
- Qué hacer con ella (ejemplos útiles):



> https://pandas.pydata.org/docs/reference/series.html

### A) Inspección / metadatos (atributos → sin paréntesis)
Atributos describen; no “hacen”. Si no lleva (), no ejecuta nada, solo devuelve información.

- s.dtype, s.dtypes (tipo/s)

- s.shape, s.size, s.ndim, s.empty

- s.index (devuelve un Index)

- s.name (nombre de la serie)

Equivalentes DataFrame: df.dtypes, df.shape, df.empty, df.index, df.columns.

In [5]:
customers["segment"].dtype

dtype('O')

### B) Resumen / conteos

 devuelven otra Serie/objeto”, no modifican s.

- s.value_counts(): conteo de valores (muy usado)

- s.unique() y s.nunique()

- s.describe() (estadísticos)

*Equivalentes DataFrame:*

- df["col"].value_counts() (lo normal)

- df.value_counts(subset=["col"]) (cuenta combinaciones de columnas)

- df.nunique() (por columna) / df.describe()



In [6]:
customers["segment"].value_counts()


segment
SMB           163
Consumer      134
MidMarket     131
Enterprise     72
Name: count, dtype: int64

### C) Selección / comparación (crean máscaras booleanas)

- s.eq(x), ne, gt, ge, lt, le

- s.isin(lista)

- s.str.contains(pat) (si es texto)

- *Patrón de filtrado*: df.loc[s.eq("APAC"), ["region"]]

*Equivalentes DataFrame*: idénticos sobre df["col"]; o en varias columnas con expresiones por columna.

 >primero creo máscara (Serie de True/False) → luego filtro con .loc[mask, cols].
- “Filtro canónico: mask = s.eq('APAC') y luego df.loc[mask, ['region']]
- “Vectoriza primero (.str, .dt, .isin, .where)

In [7]:
customers["customer_id"] != 2


0       True
1      False
2       True
3       True
4       True
       ...  
495     True
496     True
497     True
498     True
499     True
Name: customer_id, Length: 500, dtype: bool

## DATAFRAME (pd.dataframe) - tabla

- Es una tabla con 1..N columnas.

- Ej.: customers[["region"]] → DataFrame de 1 columna.

- Cuándo: selección de varias columnas, joins, groupby complejo, exportar.
>> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [8]:
customers.head() # método head ()

customers.columns
customers.index
customers.values


customers.head()



,customer_id,segment,region,channel_preferred,created_at,b2_type
0,1,Enterprise,APAC,Online,2023-04-22,B2B
1,2,Consumer,NA,Retail,2024-06-22,B2C
2,3,Consumer,NA,Online,2024-05-11,B2C
3,4,SMB,APAC,Online,2023-03-15,B2B
4,5,SMB,NA,Online,2024-08-14,B2B


### A) trocerar usando corchetes
Solo columnas → df[["col1","col2"]] (rápido de escribir). ✅ seguro  
Filas +. columnas →  df[mascara][["col1","col2"]]  ⚠️ No recomendable

In [9]:
# mascara booleana = Serie


mascara = customers["segment"] == "Enterprise"

customers[mascara][["segment", "region", "channel_preferred"]]

customers




,customer_id,segment,region,channel_preferred,created_at,b2_type
0,1,Enterprise,APAC,Online,2023-04-22,B2B
1,2,Consumer,NA,Retail,2024-06-22,B2C
2,3,Consumer,NA,Online,2024-05-11,B2C
3,4,SMB,APAC,Online,2023-03-15,B2B
4,5,SMB,NA,Online,2024-08-14,B2B
...,...,...,...,...,...,...
495,496,SMB,EU,Retail,2023-06-15,B2B
496,497,Consumer,NA,Retail,2023-07-28,B2C
497,498,SMB,APAC,Retail,2023-10-15,B2B
498,499,MidMarket,LATAM,Online,2023-11-17,B2B


### B) trocerar usando .loc 
- df.loc[Filas, Columnas]
- df.loc[mascara, Columnas]



In [31]:

customers.loc[customers["segment"] == "Enterprise",["region", "channel_preferred"]]



,region,channel_preferred
0,APAC,Online
18,LATAM,Retail
22,APAC,Online
28,LATAM,Partner
34,EU,Retail
...,...,...
458,LATAM,Online
469,EU,Online
475,LATAM,Online
476,NA,Partner
